In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import spacy
import torch.nn as nn

# Import Required Libraries & Data Loading

In [2]:
#importing the training data
df=pd.read_csv('IMDB Dataset.csv')
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Data Preparation

In [3]:
"""
sentiment : 0 = negative, 1 = positive 
use the following to get the sentiment of a sentence :  
sentiment = 0 if sentiment is negative else 1


use np.where to get the sentiment of a sentence :
"""
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [5]:
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']

In [6]:
"""
Load the spacy model and load the English language model from https://spacy.io/usage/models
"""
nlp =spacy.load("en_core_web_sm")### ADD YOUR SPACY MODEL HERE ###

/Users/bennylisangi/opt/anaconda3/lib/python3.9/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [7]:
# general Settings

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.02### ADD YOUR LEARNING RATE HERE ###
BATCH_SIZE = 10### ADD YOUR BATCH SIZE HERE ###
NUM_EPOCHS = 5## ADD YOUR NUMBER OF EPOCHS HERE ###
DEVICE =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

EMBEDDING_DIM = 500### ADD YOUR EMBEDDING DIMENSION HERE ###
HIDDEN_DIM = 50### ADD YOUR HIDDEN DIMENSION HERE ###
NUM_CLASSES = 2

cpu


In [8]:
!pip install -U torch==1.8.0 torchtext==0.9.0

# Text & label Preparation

In [9]:
# Define feature processing
"""
Define the fields for the data.
"""
TEXT = torchtext.legacy.data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')

In [10]:
# Define Label processing
LABEL = torchtext.legacy.data.LabelField(dtype = torch.long)

In [11]:
"""
Define the fields for the data.
"""

df.to_csv('moviedata.csv', index = None)
df = pd.read_csv('moviedata.csv')
df.head()

,TEXT_COLUMN_NAME,LABEL_COLUMN_NAME
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [12]:
# process the dataset

fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
                    path = "moviedata.csv", ### ADD YOUR DATASET PATH HERE ###
                    format ="CSV" , ### ADD YOUR DATASET FORMAT HERE ###
                    skip_header = False, ### ADD YOUR SKIP HEADER HERE ### 
                    fields = fields ### ADD YOUR FIELDS HERE ### 
)

# Data Split

In [13]:
# Split dataset into train and test set

train_data, test_data = dataset.split(split_ratio = [0.8, 0.2], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of test data', len(test_data))

Length of train data 40001
Length of test data 10000


In [14]:
train_data, val_data = train_data.split(split_ratio = [0.85, 0.15], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of valid data', len(val_data))

Length of train data 34001
Length of valid data 6000


# Data Observation after Tokenization

In [15]:
# Look at first traning example

print(vars(train_data.examples[2000]))

{'TEXT_COLUMN_NAME': ['Adorable', '!', 'I', 'saw', 'Domestic', 'Import', 'in', 'Philly', 'in', 'October', 'with', 'my', 'kids', '.', 'We', 'all', 'liked', 'it', 'so', 'much', 'that', 'we', 'saw', 'it', 'a', 'second', 'time', 'with', 'my', 'parents', '.', 'I', 'have', "n't", 'heard', 'them', 'laugh', 'like', 'that', 'in', 'years', '!', 'It', 'was', 'the', 'first', 'time', 'that', 'I', 'can', 'remember', 'seeing', 'a', 'movie', 'that', 'my', 'parents', 'and', 'my', 'kids', 'could', 'enjoy', '.', 'It', "'s", 'really', 'cute', 'and', 'we', 'ca', "n't", 'wait', 'for', 'it', 'to', 'come', 'out', 'on', 'DVD', '.', 'They', 'need', 'to', 'make', 'more', 'movies', 'like', 'Domestic', 'Import', '.', 'It', 'is', 'refreshing', 'to', 'go', 'to', 'a', 'movie', 'that', 'three', 'different', 'generations', 'can', 'enjoy', '(', 'and', 'not', 'be', 'embarrassed', ')', '.', 'I', 'have', 'not', 'seen', 'a', 'movie', 'this', 'cute', 'since', 'My', 'Big', 'Fat', 'Greek', 'Wedding', '.', 'I', 'loved', 'Mindy'

In [16]:
# Build Vocabulary

TEXT.build_vocab(train_data, max_size = VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'vocabulary size: {len(TEXT.vocab)}')
print(f'Label Size: {len(LABEL.vocab)}')

vocabulary size: 20002
Label Size: 3


 2 extra value in vocabulary is because added (unknown) and (padding)

In [17]:
# Print the most common words: Use the most_common method of the TEXT vocabulary
most_common_words = TEXT.vocab.freqs.most_common(50)
print(most_common_words)

[('the', 392110), (',', 371221), ('.', 319879), ('a', 210542), ('and', 210316), ('of', 195116), ('to', 180616), ('is', 146429), ('in', 118677), ('I', 106550), ('it', 103775), ('that', 94301), ('"', 86397), ("'s", 82967), ('this', 82101), ('-', 71087), ('/><br', 69261), ('was', 67888), ('as', 58025), ('movie', 57914), ('with', 57762), ('for', 56443), ('film', 52340), ('The', 51301), ('but', 46914), ("n't", 44701), ('on', 44630), ('(', 44449), (')', 43708), ('you', 42067), ('are', 40610), ('not', 38989), ('have', 38503), ('his', 37675), ('be', 36184), ('one', 32993), ('he', 32886), ('!', 30128), ('at', 29700), ('by', 29439), ('all', 28586), ('an', 28284), ('who', 27647), ('they', 26484), ('from', 26421), ('like', 26362), ('so', 23874), ('or', 23233), ('about', 22983), ('has', 22730)]


In [18]:
# Token corresponding to first 10 Indices

print(TEXT.vocab.itos[:20]) #itos = Integer to string

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is', 'in', 'I', 'it', 'that', '"', "'s", 'this', '-', '/><br', 'was']


# Data Preparation for Batch wise Implimentation

In [19]:
# Define Dataloader

train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data), ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
        batch_size = BATCH_SIZE, ### ADD YOUR BATCH SIZE HERE ###
        sort_within_batch = True, ### ADD YOUR SORT WITHIN BATCH HERE ### 
        sort_key = lambda x : len(x.TEXT_COLUMN_NAME), 
        #device = DEVICE
    )

In [20]:
# Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([255, 10])
Target vector size: torch.Size([10])

Valid:
Text matrix size: torch.Size([32, 10])
Target vector size: torch.Size([10])

Test:
Text matrix size: torch.Size([28, 10])
Target vector size: torch.Size([10])


In [21]:
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data), ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
        batch_size = BATCH_SIZE, ### ADD YOUR BATCH SIZE HERE ###
        sort_within_batch = True, ### ADD YOUR SORT WITHIN BATCH HERE ###
        sort_key = lambda x : len(x.TEXT_COLUMN_NAME),
        device = DEVICE
    )

# Model Building

In [22]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super(RNN, self).__init__()
        ### ADD YOUR CODE HERE ###
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        ### END YOUR CODE ### 

    def forward(self, text):
        ### ADD YOUR CODE HERE ###
        # text dim: [sentence length, batch size]
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]
        
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]
        output, hidden = self.rnn(embedded)
        # hidden dim: [batch size, hidden dim]
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        out = self.fc(hidden.squeeze(0))
        
        ### END YOUR CODE ###
        return out

In [23]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab), ### ADD YOUR INPUT DIM HERE. This can be the length of your vocabulary or the embedding dim ###
            embedding_dim= EMBEDDING_DIM, ### ADD YOUR EMBEDDING DIM HERE ###
            hidden_dim=HIDDEN_DIM, ### ADD YOUR HIDDEN DIM HERE ###
            output_dim=NUM_CLASSES  ### ADD NUMBER OF CLASSES HERE ###
)

model = model.to(DEVICE)
optimizer =torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) ### ADD YOUR OPTIMIZER HERE ###
criterion = nn.BCELoss()
criterio = criterion.to(DEVICE)

# Define Accuracy

In [24]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [25]:
model

RNN(
  (embedding): Embedding(20002, 500)
  (rnn): RNN(500, 50)
  (fc): Linear(in_features=50, out_features=2, bias=True)
)

# Model Run

In [ ]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        optimizer.zero_grad()
        predictions = model(text).squeeze(0)
        loss = criterion(predictions, labels)
        
        ### UPDATE MODEL PARAMETERS
        loss.backward()
        optimizer.step()
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

# Model Testing

In [ ]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][1].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

In [ ]:
print('Probability positive:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")